In [37]:
import pandas as pd
import numpy as np

# a) Generate a Dataset

We require a dataset of randomly generated labels. There should be 8 features and 50,000 examples.

Randomly generate an array of integers from 0-2

In [52]:
examples=50000
features=8
num_categories=3

rand_arr = np.ceil(np.random.rand(examples,features)*num_categories).astype(int)-1
rand_arr

array([[1, 2, 0, ..., 2, 0, 2],
       [2, 0, 0, ..., 1, 2, 0],
       [0, 2, 1, ..., 1, 1, 0],
       ...,
       [2, 0, 1, ..., 2, 2, 0],
       [1, 2, 1, ..., 1, 0, 0],
       [1, 1, 0, ..., 0, 0, 2]])

Convert this to a Pandas dataframe.

In [53]:
df = pd.DataFrame(rand_arr)
df

,0,1,2,3,4,5,6,7
0,1,2,0,1,2,2,0,2
1,2,0,0,2,2,1,2,0
2,0,2,1,0,2,1,1,0
3,1,0,2,1,1,1,2,0
4,2,1,1,1,1,0,0,0
...,...,...,...,...,...,...,...,...
49995,1,1,1,0,2,2,2,0
49996,0,0,1,1,2,0,1,1
49997,2,0,1,2,1,2,2,0
49998,1,2,1,2,2,1,0,0


In [71]:
for val in range(0,3):
    print(f"Category = {val}")
    print(df[df == val].count())

Category = 0
0    16850
1    16610
2    16590
3    16661
4    16756
5    16768
6    16768
7    16553
dtype: int64
Category = 1
0    16640
1    16770
2    16728
3    16832
4    16634
5    16589
6    16434
7    16768
dtype: int64
Category = 2
0    16510
1    16620
2    16682
3    16507
4    16610
5    16643
6    16798
7    16679
dtype: int64


In [64]:
df == 0

,0,1,2,3,4,5,6,7
0,False,False,True,False,False,False,True,False
1,False,True,True,False,False,False,False,True
2,True,False,False,True,False,False,False,True
3,False,True,False,False,False,False,False,True
4,False,False,False,False,False,True,True,True
...,...,...,...,...,...,...,...,...
49995,False,False,False,True,False,False,False,True
49996,True,True,False,False,False,True,False,False
49997,False,True,False,False,False,False,False,True
49998,False,False,False,False,False,False,True,True
